参考URLhttps://dse-souken.com/2021/03/25/ai-20/

In [ ]:
#colab_note上で必要
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [ ]:
#参考URL: https://developers.microad.co.jp/entry/2019/05/10/180000
#疎行列のpivotテーブルを作成
#index_listとcol_list
def make_sparse_pivot(df, index, col, val, index_list = [], col_list = []): 
  if(len(index_list) == 0):
    index_list = df[index].unique()
  if(len(col_list) == 0):
    col_list = df[col].unique()
  index_categorical = pd.api.types.CategoricalDtype(categories=sorted(index_list), ordered=True)
  col_categorical = pd.api.types.CategoricalDtype(categories=sorted(col_list), ordered=True)
  row_num = df[index].astype(index_categorical).cat.codes
  col_num = df[col].astype(col_categorical).cat.codes
  
  sparse_matrix = csr_matrix((df[val], (row_num, col_num)), \
                           shape=(index_categorical.categories.size, \
                                  col_categorical.categories.size))
  return sparse_matrix

In [ ]:
folder_path = '/content/drive/My Drive/dressipi_recsys2022/'
file_name_list = ["candidate_items.csv", "item_features.csv", "test_final_sessions.csv", "test_leaderboard_sessions.csv", "train_purchases.csv", "train_sessions.csv"]
df_candidate_items = pd.read_csv(folder_path + file_name_list[0])
df_item_features = pd.read_csv(folder_path + file_name_list[1])
df_test_final_sessions = pd.read_csv(folder_path + file_name_list[2])
df_test_leaderboard_sessions = pd.read_csv(folder_path + file_name_list[3])
df_train_purchases = pd.read_csv(folder_path + file_name_list[4])
df_train_sessions = pd.read_csv(folder_path + file_name_list[5])

In [ ]:
#pivot用のカラムを作成
df_train_sessions['count'] = 1.0
df_test_leaderboard_sessions['count'] = 1.0

In [ ]:
#この方法では最終的にcandidate itemが買われていないセッションの使い道はないので予めデータを落としておく
candidate_items = df_candidate_items['item_id'].unique()
candidate_session_list = df_train_purchases[df_train_purchases['item_id'].isin(candidate_items)]['session_id'].values
df_train_purchases = df_train_purchases[df_train_purchases['session_id'].isin(candidate_session_list)]
df_train_sessions = df_train_sessions[df_train_sessions['session_id'].isin(candidate_session_list)]

In [ ]:
#trainデータとして全trainセッションを使用
df_train = df_train_sessions

In [ ]:
#各セッションデータを疎行列に変換
index = 'session_id'
col = 'item_id'
val = 'count'
col_list = df_item_features['item_id'].unique()

train_sparse = make_sparse_pivot(df_train_sessions, index, col, val, col_list = col_list)
lb_session_sparse = make_sparse_pivot(df_test_leaderboard_sessions, index, col, val, col_list = col_list)
#pandasでpivotすると落ちる
#df_piv = df_train_sessions.pivot_table(index= "session_id", columns="item_id" ,values="date", aggfunc = "count").fillna(0)

In [ ]:
#KNNのインスタンス化。評価方法をコサイン類似度に設定。bruteは総当り方式。
rec = NearestNeighbors(n_neighbors=10,algorithm= "brute", metric= "cosine")
# KNNで訓練
rec_model = rec.fit(train_sparse)

In [ ]:
unique_session_lb = df_test_leaderboard_sessions['session_id'].unique()
unique_item_id_list = df_item_features['item_id'].unique()
df_train_purchases = df_train_purchases.set_index('session_id')
session_id_list = df_train_purchases.index

In [ ]:
import warnings
warnings.simplefilter('ignore')

#距離が近い上位10000までのセッションのみを最終的な
num_n_neighbors = 10000

#予測：雑実装のため60分くらいかかる、、、
df_sub_lb = pd.DataFrame(columns=['session_id', 'item_id', 'rank'])
item_id = []
session_id = []
rank = []
count = 0

for i,s  in enumerate(unique_session_lb):

  #for文のカウント,50000で終わり
  if(count%1000 == 0):
    print(count)
  count += 1

#対象のlb sessionとの距離が近い、上位num_n_neighborsのtrain session の、indexと距離を取得
  distance, indice = rec_model.kneighbors(lb_session_sparse.getrow(i).toarray().flatten().reshape(1,-1),n_neighbors=num_n_neighbors)
  df_distance = pd.DataFrame(np.array([df_train_purchases.loc[session_id_list[indice].flatten()].item_id.values, 1 - distance[0]]).T, columns=['candidate_item_id', 'distance'])

#そのsessionに登場したアイテムを除く
  session_item_list = df_test_leaderboard_sessions[df_test_leaderboard_sessions['session_id']==s].item_id.values
  df_distance = df_distance[df_distance['candidate_item_id'].isin(session_item_list)==False]

#距離をもとにした重みで、アイテムごとの加重平均をとる->スコアが高い上位100アイテムを取得
  prob_top100_item_id=df_distance.groupby('candidate_item_id').sum().sort_values('distance', ascending = 0).index.astype('int').values[0:100]

#結果を格納していく
  item_id = np.concatenate([item_id,prob_top100_item_id])

  temp = np.array([s for i in range(len(prob_top100_item_id))])
  session_id = np.concatenate([session_id,temp])

  temp = np.array([i + 1 for i in range(len(prob_top100_item_id))])
  rank = np.concatenate([rank,temp])


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [ ]:
#提出用のデータを作成
df_sub_lb = pd.DataFrame(np.vstack([session_id, item_id, rank]).T, columns=['session_id', 'item_id', 'rank'])
df_sub_lb = df_sub_lb.astype('int')#intに明示的に変換しないと通らない

In [ ]:
#保存
file_name = "submit20220425_cos_dist3"
df_sub_lb.to_csv(folder_path + file_name + ".csv", index=False)

以下はゴミ

In [ ]:

df_test_leaderboard_sessions[df_test_leaderboard_sessions['session_id']==1402710]

session_id  item_id                     date  count
72746     1402710    21668  2021-06-22 12:36:15.701    1.0
72747     1402710    25809  2021-06-22 12:38:16.523    1.0
72748     1402710    13656  2021-06-22 12:38:34.615    1.0
72749     1402710      955  2021-06-22 12:39:28.532    1.0
72750     1402710     6736   2021-06-22 12:39:32.15    1.0

In [ ]:
df_sub_lb.isnull().all()

session_id    False
item_id       False
rank          False
dtype: bool

In [ ]:
pd.set_option('display.max_rows', 200)
print(df_sub_lb[df_sub_lb['session_id']==1402710])

         session_id  item_id  rank
1570400     1402710    26853     1
1570401     1402710     8577     2
1570402     1402710    15249     3
1570403     1402710    21616     4
1570404     1402710    21890     5
1570405     1402710    18289     6
1570406     1402710    26542     7
1570407     1402710    24736     8
1570408     1402710    12662     9
1570409     1402710    11998    10
1570410     1402710     8861    11
1570411     1402710      447    12
1570412     1402710     6627    13
1570413     1402710    14648    14
1570414     1402710    23428    15
1570415     1402710    20798    16
1570416     1402710     1933    17
1570417     1402710    21965    18
1570418     1402710    15777    19
1570419     1402710    18947    20
1570420     1402710      340    21
1570421     1402710     3857    22
1570422     1402710    21100    23
1570423     1402710     6256    24
1570424     1402710    24286    25
1570425     1402710     2691    26
1570426     1402710     4193    27
1570427     1402710 

In [ ]:
len(df_sub_lb)

5000000

In [ ]:
test= df_item_features['item_id'].unique()

In [ ]:
candidate_items = df_candidate_items['item_id'].unique()

In [ ]:
df_sub_lb[df_sub_lb['item_id'].isin(candidate_items)]

session_id  item_id  rank
0                26     3260     1
1                26    26538     2
2                26    20541     3
3                26     5383     4
4                26    27416     5
...             ...      ...   ...
4999995     4439757     4564    96
4999996     4439757    17750    97
4999997     4439757    21886    98
4999998     4439757    10004    99
4999999     4439757    14680   100

[5000000 rows x 3 columns]

In [ ]:
df_sub_lb[df_sub_lb['item_id'].isin(candidate_items)].head(100)

session_id  item_id  rank
0           26     3260     1
1           26    26538     2
2           26    20541     3
3           26     5383     4
4           26    27416     5
5           26     6603     6
6           26    21035     7
7           26     3425     8
8           26    12939     9
9           26     2213    10
10          26     7019    11
11          26    27852    12
12          26    25166    13
13          26    19480    14
14          26    16194    15
15          26    14040    16
16          26     6021    17
17          26    24801    18
18          26     7719    19
19          26    25300    20
20          26    27921    21
21          26    21624    22
22          26    21928    23
23          26     1828    24
24          26    20883    25
25          26    16600    26
26          26    15911    27
27          26     7771    28
28          26    20889    29
29          26     1483    30
30          26    11343    31
31          26    27182    32
32          26    18093    33
33          26     6237    34
34          26      823    35
35          26    21965    36
36          26     1107    37
37          26    26160    38
38          26    25188    39
39          26    20108    40
40          26    16806    41
41          26     7368    42
42          26    19021    43
43          26    17231    44
44          26      934    45
45          26    16941    46
46          26    26868    47
47          26      628    48
48          26     9891    49
49          26      452    50
50          26     9833    51
51          26     6680    52
52          26    19956    53
53          26     9018    54
54          26     2672    55
55          26    21578    56
56          26    27545    57
57          26    12555    58
58          26    19713    59
59          26     8577    60
60          26    17514    61
61          26    10236    62
62          26     4267    63
63          26    14201    64
64          26     5674    65
65          26    21097    66
66          26      320    67
67          26    27888    68
68          26    13310    69
69          26    27630    70
70          26    23572    71
71          26    22896    72
72          26    23612    73
73          26     6373    74
74          26    26124    75
75          26    22669    76
76          26    14136    77
77          26    16367    78
78          26    10475    79
79          26    26108    80
80          26    21900    81
81          26    25489    82
82          26    16668    83
83          26        9    84
84          26     1578    85
85          26    11524    86
86          26    21278    87
87          26     4138    88
88          26     3209    89
89          26    16299    90
90          26    19115    91
91          26    19071    92
92          26    19048    93
93          26    19058    94
94          26    19060    95
95          26    14612    96
96          26    19064    97
97          26    19066    98
98          26    19087    99
99          26    19112   100

過去のやつ

In [ ]:
import warnings
warnings.simplefilter('ignore')

#予測：雑実装のため60分くらいかかる、、、
df_sub_lb = pd.DataFrame(columns=['session_id', 'item_id', 'rank'])
item_id = []
session_id = []
rank = []
count = 0
for i,s  in enumerate(unique_session_lb):
  #for文のカウント,50000で終わり
  if(count%1000 == 0):
    print(count)
  count += 1

  distance, indice = rec_model.kneighbors(lb_session_sparse.getrow(i).toarray().flatten().reshape(1,-1),n_neighbors=300)
  candidate_df = df_train_purchases.loc[session_id_list[indice].flatten()]
  prob_top100_item_id = candidate_df[candidate_df['item_id'].isin(lb_item_id_list)]['item_id'].unique()[0:100]
  if(len(prob_top100_item_id) != 100):
    distance, indice = rec_model.kneighbors(lb_session_sparse.getrow(i).toarray().flatten().reshape(1,-1),n_neighbors=1000)
    candidate_df = df_train_purchases.loc[session_id_list[indice].flatten()]
    prob_top100_item_id = candidate_df[candidate_df['item_id'].isin(lb_item_id_list)]['item_id'].unique()[0:100]
  item_id = np.concatenate([item_id,prob_top100_item_id])

  temp = np.array([s for i in range(len(prob_top100_item_id))])
  session_id = np.concatenate([session_id,temp])

  temp = np.array([i + 1 for i in range(len(prob_top100_item_id))])
  rank = np.concatenate([rank,temp])

映画ID100を見た人におすすめの映画IDは以下です。


IndexError: ignored

In [ ]:
def get_similairty(person1, person2):

  ## 両者とも見た映画の集合を取る
  set_person1 = set(dataset[person1].keys())
  set_person2 = set(dataset[person2].keys())
  set_both = set_person1.intersection(set_person2)

  if len(set_both)==0: #共通でみた映画がない場合は類似度を0とする
    return 0

  list_destance = []

  for item in set_both:
    # 同じ映画のレビュー点の差の2乗を計算
    # この数値が大きいほど「気が合わない」=「似ていない」と定義できる 
    distance = pow(dataset[person1][item]-dataset[person2][item], 2) 
    list_destance.append(distance)

  return 1/(1+np.sqrt(sum(list_destance))) #各映画の気の合わなさの合計の逆比的な指標を返す

In [ ]:
get_similairty('Lisa Rose','Jack Matthews')

0.3405424265831667

In [ ]:
def get_recommend(person, top_N):

  totals = {} ; simSums = {} #推薦度スコアを入れるための箱を作っておく

  # 自分以外のユーザのリストを取得してFor文を回す
  # -> 各人との類似度、及び各人からの（まだ本人が見てない）映画の推薦スコアを計算するため
  list_others = dataset.keys() ; list_others.remove(person)

  for other in list_others:
    # 本人がまだ見たことが無い映画の集合を取得
    set_other = set(dataset[other]); set_person = set(dataset[person])
    set_new_movie = set_other.difference(set_person)

    # あるユーザと本人の類似度を計算(simは0~1の数字)
    sim = get_similairty(person, other)

    # (本人がまだ見たことがない)映画のリストでFor分を回す
    for item in set_new_movie:

      # "類似度 x レビュー点数" を推薦度のスコアとして、全ユーザで積算する
      totals.setdefault(item,0)
      totals[item] += dataset[other][item]*sim 

      # またユーザの類似度の積算値をとっておき、これで上記のスコアを除する
      simSums.setdefault(item,0)
      simSums[item] += sim

  rankings = [(total/simSums[item],item) for item,total in totals.items()]
  rankings.sort()
  rankings.reverse()

  return [i[1] for i in rankings][:top_N]

In [ ]:
get_recommend('Toby',2)

AttributeError: ignored

In [ ]:
!git clone https://github.com/yolo-kiyoshi/recommend_cf.git


Cloning into 'recommend_cf'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 28 (delta 5), reused 25 (delta 2), pack-reused 0
Unpacking objects: 100% (28/28), done.


In [ ]:
import os
path = '/content/recommend_cf'

#作業ディレクトリをpathに移動する
os.chdir(path)

#作業ディレクトリ直下のファイルを確認
!ls

data		    Dockerfile	poetry.lock	README.md  tests
docker-compose.yml  notebook	pyproject.toml	src
